<a href="https://colab.research.google.com/github/rrankawat/stm32/blob/main/MNIST_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trained Model

In [1]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Data Loaders
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_transform = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_transform = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_transform, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_transform, batch_size=batch_size, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 12.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 335kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.14MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.00MB/s]


In [3]:
# -----------------------
# Model
# -----------------------
class MNISTTinyCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
    self.conv2 = nn.Conv2d(6, 16, 3, padding=1)
    self.fc1 = nn.Linear(16 * 7 * 7, 196)
    self.fc2 = nn.Linear(196, 49)
    self.fc3 = nn.Linear(49, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2) # 28 / 2 -> 14

    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2) # 14 / 2 -> 7

    x = x.view(x.size(0), -1) # Flatten

    x = F.relu(self.fc1(x)) # 784 -> 196
    x = F.relu(self.fc2(x)) # 196 -> 49
    x = self.fc3(x) # 49 -> 10 (logits)

    return x

In [4]:
# Random Seeds and Model Instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = MNISTTinyCNN().to(device)

In [5]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training
def train(epoch, model, loader, criterion, optimizer):
  model.train() # training mode

  correct = 0
  total = 0
  running_loss = 0

  for b, (X, y) in enumerate(loader):
    X = X.to(device)
    y = y.to(device)

    # Forward
    outputs = model(X)

    # Loss
    loss = criterion(outputs, y)

    # Backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Metrics
    batch_size = y.size(0)
    running_loss += loss.item() * batch_size
    preds = outputs.argmax(dim=1)
    correct += (preds == y).sum().item()
    total += batch_size

  avg_loss = running_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [7]:
# Testing
def test(model, loader, criterion):
  model.eval() # testing mode

  correct = 0
  total = 0
  running_loss = 0

  with torch.no_grad():
    for X, y in loader:
      X = X.to(device)
      y = y.to(device)

      # Forward
      outputs = model(X)

      # Loss
      loss = criterion(outputs, y)

      # Metrics
      batch_size = y.size(0)
      running_loss += loss.item() * batch_size
      preds = outputs.argmax(dim=1)
      correct += (preds == y).sum().item()
      total += batch_size

  avg_loss = running_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [8]:
epochs = 5
start_time = time.time()

for epoch in range(1, epochs + 1):
  train_loss, train_acc = train(epoch, model, train_loader, criterion, optimizer)
  test_loss, test_acc = test(model, test_loader, criterion)

  print(
      f"Epoch {epoch}/{epochs} | "
      f"Train loss: {train_loss:.4f}, Train acc: {train_acc*100:.2f}% | "
      f"Test loss: {test_loss:.4f}, Test acc: {test_acc*100:.2f}%"
  )

print(f"Time taken: {(time.time() - start_time) / 60} minutes!")

Epoch 1/5 | Train loss: 0.2322, Train acc: 93.04% | Test loss: 0.0706, Test acc: 97.73%
Epoch 2/5 | Train loss: 0.0648, Train acc: 97.98% | Test loss: 0.0455, Test acc: 98.55%
Epoch 3/5 | Train loss: 0.0451, Train acc: 98.59% | Test loss: 0.0325, Test acc: 98.84%
Epoch 4/5 | Train loss: 0.0323, Train acc: 99.00% | Test loss: 0.0368, Test acc: 98.75%
Epoch 5/5 | Train loss: 0.0264, Train acc: 99.15% | Test loss: 0.0389, Test acc: 98.88%
Time taken: 2.6832316676775614 minutes!


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/stm_mnist_model.pth")
print("✅ Model saved as stm_mnist_model.pth")

✅ Model saved as stm_mnist_model.pth
